In [38]:
import os
import json
import pandas as pd
import traceback

# https://python.langchain.com/docs/integrations/chat/openai
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [39]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [41]:
# Initialize ChatopenAI
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo",temperature=0.5)

In [43]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import  SequentialChain
from langchain.callbacks import get_openai_callback

In [55]:
# JSON response for the chatbot
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct_answer"
    },
    "5": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    }
}

In [73]:
# Template 1 for the prompt
TEMPLATE ="""
text:{text}
You are an expert MCQ maker. Give the above text, it is your job to\
create a quize of {number} multiple choice questions for {subject} in a {tone} tone\
Make sure questions are not repeated and check all question to be confirming to the text as well. \
Make sure to format your resepomse like RESPONSE_JSON below and use it as a guide.  \
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}

"""

In [74]:
# Quiz generation prompt
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [75]:
# Quiz generation chain
quize_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [76]:
# Template 2 for the prompt
TEMPLATE2="""
Your are an expert English grammerian. Give a multiple choice quiz for {subject} students. \
    you need to evaluate the complexity of the question and give a complete analysis of the quiz. 
    Only use at max 50 words for the complexity analysis. \
    If the questions are not in par with the student's cognitive and analytical abilities of the student, \
    Update the questions which needs to be changed and change the tone such that it perfectly fits the student's cognitive and analytical abilities. \
    Quiz_MCQs: 
    {quiz}

    check from an expert English writer of the above quiz:
    """

In [77]:
# Question evaluation prompt
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [78]:
# Quiz evaluation chain
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [80]:
# Sequential chain for quiz generation and evaluation
generate_evaluate_chain = SequentialChain(  
    chains=[quize_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)